In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Stopwords

In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
len(STOP_WORDS)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

326

In [2]:
nlp=spacy.load('en_core_web_sm')
doc=nlp("We just opened our wings , the flying part is coming soon.")

for token in doc:
    if token.is_stop:
        print(token)

We
just
our
the
part
is


In [4]:
def preprocess(text):
    nlp=spacy.load('en_core_web_sm')
    doc=nlp(text)
    
    not_stopwords=[]
    for token in doc:
        if token.is_stop | token.is_punct:
            pass
        else: not_stopwords.append(str(token))
    return " ".join(not_stopwords)
            

In [5]:
preprocess("We just opened our wings , the flying part is coming soon.")

'opened wings flying coming soon'

## n- grams

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
v= CountVectorizer(ngram_range=(1,3))
v.fit(["Thor Hathodawala is looking for a job"])

v.vocabulary_

{'thor': 12,
 'hathodawala': 2,
 'is': 5,
 'looking': 9,
 'for': 0,
 'job': 8,
 'thor hathodawala': 13,
 'hathodawala is': 3,
 'is looking': 6,
 'looking for': 10,
 'for job': 1,
 'thor hathodawala is': 14,
 'hathodawala is looking': 4,
 'is looking for': 7,
 'looking for job': 11}

In [8]:
corpus=[
    "thor ate pizza",
    "loki is tall",
    "loki is eating pizza"
]

In [13]:
import spacy

nlp=spacy.load('en_core_web_sm')

def preprocess(text):
    doc=nlp(text)
    
    filtered_words=[]
    for token in doc:
        if token.is_stop | token.is_punct:
            pass
        else: filtered_words.append(str(token.lemma_))
    return " ".join(filtered_words)
    

In [14]:
preprocess("loki is eating pizza")

'loki eat pizza'

In [15]:
corpus_processed= [ preprocess(text) for text in corpus]
corpus_processed

['thor eat pizza', 'loki tall', 'loki eat pizza']

In [16]:
v= CountVectorizer(ngram_range=(1,2))
v.fit(corpus_processed)

v.vocabulary_

{'thor': 7,
 'eat': 0,
 'pizza': 5,
 'thor eat': 8,
 'eat pizza': 1,
 'loki': 2,
 'tall': 6,
 'loki tall': 4,
 'loki eat': 3}

In [17]:
v.transform(["thor eat pizza"]).toarray()

array([[1, 1, 0, 0, 0, 1, 0, 1, 1]])

In [18]:
v.transform(["hulk eat pizza"]).toarray()   # out of vocabulary problem

array([[1, 1, 0, 0, 0, 1, 0, 0, 0]])